- 各クラスタ内の重心ベクトルとoutlaierベクトルを取得して固有名詞ベクトルを制限する
- 関係ベクトルを算出する
- 関係ベクトルにをランダムで20%ずつ取得し、それぞれのクラスタを求める

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
import random

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
from sklearn.cluster import DBSCAN
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [37]:
f = open('../koyuu_noun_sample_list.binaryfile','rb')
sample_vec_list = pickle.load(f)


In [38]:
# データ取得
#eps minPts n_class

file = "../dbscan_koyuu_nomal_noun_fold/0.5_10_dic.binaryfile"
f = open(file,'rb')
eps_minPts_dic = pickle.load(f)

for eps, minPts_data in eps_minPts_dic.items():
    for minPts, y_dbscan in minPts_data.items():
            
        n_class = np.max(y_dbscan) +1
        n_outlier = len(np.where(y_dbscan == -1)[0])
                
        print(str(eps) +" "+ str(minPts) +" "+ str(n_class) +" " + str(n_outlier))

0.5 2 643 33999
0.5 4 104 35525
0.5 6 58 36312
0.5 8 41 36845
0.5 10 28 37302


In [39]:
y_dbscan = eps_minPts_dic[0.5][4]

In [40]:
# クラスとそのクラス内のベクトル数

n_class = np.max(y_dbscan) +1

for n in range(n_class):
    print("class", n)
    class_n =  len(np.where(y_dbscan == n)[0])
    print(class_n)

class 0
6369
class 1
36
class 2
5
class 3
108
class 4
5
class 5
19
class 6
125
class 7
14
class 8
109
class 9
15
class 10
13
class 11
7
class 12
9
class 13
54
class 14
21
class 15
143
class 16
24
class 17
7
class 18
7
class 19
22
class 20
6
class 21
4
class 22
27
class 23
12
class 24
56
class 25
11
class 26
14
class 27
12
class 28
10
class 29
4
class 30
26
class 31
5
class 32
4
class 33
17
class 34
5
class 35
21
class 36
10
class 37
8
class 38
46
class 39
4
class 40
28
class 41
4
class 42
19
class 43
4
class 44
8
class 45
7
class 46
22
class 47
5
class 48
6
class 49
4
class 50
10
class 51
6
class 52
6
class 53
5
class 54
8
class 55
19
class 56
9
class 57
5
class 58
4
class 59
4
class 60
3
class 61
7
class 62
3
class 63
12
class 64
6
class 65
5
class 66
4
class 67
5
class 68
7
class 69
5
class 70
5
class 71
5
class 72
3
class 73
9
class 74
4
class 75
5
class 76
4
class 77
10
class 78
7
class 79
5
class 80
5
class 81
7
class 82
4
class 83
5
class 84
4
class 85
4
class 86
4
class 87
8
cla

In [41]:
max_n = 1
mean_vec_list = [] # 関係ベクトルの配列

In [42]:
for n in range(n_class):
    print("class", n)
    class_number =  np.where(y_dbscan == n)[0]
    # クラスタに含まれる点の数
    #print("クラスタ" + str(n) +  "に含まれる点の数", len(np.where(data == n)[0]))
            
    index = list(np.where(y_dbscan == n))
    index =list(index[0])
    #print(index)
            
    d = np.array([sample_vec_list[i] for i in index])
        
    #クラスタの重心
    mean = np.mean(d, axis=0)
    mean_vec_list.append(mean)    

class 0
class 1
class 2
class 3
class 4
class 5
class 6
class 7
class 8
class 9
class 10
class 11
class 12
class 13
class 14
class 15
class 16
class 17
class 18
class 19
class 20
class 21
class 22
class 23
class 24
class 25
class 26
class 27
class 28
class 29
class 30
class 31
class 32
class 33
class 34
class 35
class 36
class 37
class 38
class 39
class 40
class 41
class 42
class 43
class 44
class 45
class 46
class 47
class 48
class 49
class 50
class 51
class 52
class 53
class 54
class 55
class 56
class 57
class 58
class 59
class 60
class 61
class 62
class 63
class 64
class 65
class 66
class 67
class 68
class 69
class 70
class 71
class 72
class 73
class 74
class 75
class 76
class 77
class 78
class 79
class 80
class 81
class 82
class 83
class 84
class 85
class 86
class 87
class 88
class 89
class 90
class 91
class 92
class 93
class 94
class 95
class 96
class 97
class 98
class 99
class 100
class 101
class 102
class 103


In [43]:
len(mean_vec_list)

104

In [44]:
class_number =  np.where(y_dbscan == -1)[0]
index = list(np.where(y_dbscan == -1))
index =list(index[0])
d = np.array([sample_vec_list[i] for i in index])

for v in d:
    mean_vec_list.append(v)

In [45]:
len(mean_vec_list)

35629

# 関係ベクトルを算出する

In [ ]:
# 関係ベクトル出力

relation_vec_list = []

for start in range(len(mean_vec_list)):
    f_name = "../relation_vec_fold/" + str(start) + ".binaryfile"
    
    for i in range(start+1, len(mean_vec_list)):
            relarion_vec = mean_vec_list[start] - mean_vec_list[i] 
            relation_vec_list.append(relarion_vec) 
            
f = open(f_name,'wb')
pickle.dump(relation_vec_list,f)
f.close
relation_vec_list = []

In [ ]:
# 密なクラスタを算出


def generate_mean(i):
    # 全体の固有名詞の内、半分をランダムに取得

    sample_list = random.sample(mean_vec_list, int(len(mean_vec_list)/10))

    eps_minPts = {}
    minPts_data = {}

    print("noun_vec_list数 : ", len(sample_list))

    for eps in range(1,10,1):
        eps = eps/10
    
        for minPts in range(2,12, 2):
            dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(sample_list)
        
            y_dbscan = dbscan.labels_
        
            print("y_dbscan", y_dbscan)
        
            print("eps:",eps,",minPts:", minPts)
        
            # outlier数
            print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
            # クラスタ数
            print("クラスタ数", np.max(y_dbscan) +1)
        
            print("                            ")
        
            minPts_data[minPts] = y_dbscan
            eps_minPts[eps] = minPts_data
        
        # DBSCANの結果を保存
        f_name = str(i)_str(eps) + "_" + str(minPts) + "_dic.binaryfile"
        save_at = "../0_1_dbscan_sample_mean_fold/" + f_name
        print(save_at)
        f = open(save_at,'wb')
        pickle.dump(eps_minPts,f)
        f.close
    
        eps_minPts = {}
        minPts_data = {}

In [ ]:
for i in range(50): #50*N(パラメータの生成クラス数)クラスできる
    generate_mean(i)